In [ ]:
The actual sandbox, where you explore to figure out why your code is wrong. :/

In [20]:
include("nn_ops.jl")

NNOps

In [21]:
batch = 1
in_height = 4
in_width = 4

in_channels = 5
filter_height = 2
filter_width = 2
out_channels = 3

filter = rand(-10:10, filter_height, filter_width, in_channels, out_channels)
x_actual = rand(-10:10, batch, in_height, in_width, in_channels)

1×4×4×5 Array{Int64,4}:
[:, :, 1, 1] =
 2  -7  4  -4

[:, :, 2, 1] =
 3  7  0  6

[:, :, 3, 1] =
 10  -4  -5  0

[:, :, 4, 1] =
 -8  8  -5  -10

[:, :, 1, 2] =
 8  -2  -9  10

[:, :, 2, 2] =
 -7  0  1  8

[:, :, 3, 2] =
 5  -4  8  1

[:, :, 4, 2] =
 -5  -2  8  4

[:, :, 1, 3] =
 -4  7  6  -6

[:, :, 2, 3] =
 2  -1  -2  8

[:, :, 3, 3] =
 -4  -9  -3  -10

[:, :, 4, 3] =
 -8  0  -1  1

[:, :, 1, 4] =
 5  -8  9  5

[:, :, 2, 4] =
 8  8  4  6

[:, :, 3, 4] =
 -5  5  4  -3

[:, :, 4, 4] =
 0  1  8  6

[:, :, 1, 5] =
 -10  -5  3  -2

[:, :, 2, 5] =
 9  3  -1  -6

[:, :, 3, 5] =
 -5  6  -2  -10

[:, :, 4, 5] =
 6  -3  -7  5

In [22]:
filter

2×2×5×3 Array{Int64,4}:
[:, :, 1, 1] =
 4  -6
 9   9

[:, :, 2, 1] =
 -6   5
 -4  -2

[:, :, 3, 1] =
 -2  -3
  5  -7

[:, :, 4, 1] =
   6  -5
 -10   1

[:, :, 5, 1] =
  9  -4
 -8  -8

[:, :, 1, 2] =
 -5  -4
  5  -5

[:, :, 2, 2] =
 10  8
  0  0

[:, :, 3, 2] =
 -9  6
  5  7

[:, :, 4, 2] =
 -6  -10
 -6   -8

[:, :, 5, 2] =
  3  -6
 10  -5

[:, :, 1, 3] =
   7  1
 -10  2

[:, :, 2, 3] =
 -7  3
  8  8

[:, :, 3, 3] =
  2  5
 -9  8

[:, :, 4, 3] =
 -4  5
  9  0

[:, :, 5, 3] =
 2  4
 0  6

In [23]:
output_size = (batch, in_height, in_width, out_channels)

(1,4,4,3)

In [26]:
output = Array{Real}(output_size)
filter_height_offset = round(Int, filter_height/2, RoundUp)
filter_width_offset = round(Int, filter_width/2, RoundUp)

input=x_actual

using Base.Cartesian
    @nloops 4 i output begin
        s = 0
        @nloops 4 j filter begin
            x = i_2 + j_1 - filter_height_offset
            y = i_3 + j_2 - filter_width_offset
            if x > 0 && y > 0 && x<=in_height && y<=in_width
                # Doing bounds check to make sure that we stay within bounds
                # for input. This effectively zero-pads the input.
                # TODO: Use default checkbounds function here instead?
                # TODO: Addition here is a bottleneck; figure out whether
                # you could use append without making this incompatible
                # with normal numbers
                s += input[i_1, x, y, j_3] * filter[j_1, j_2, j_3, i_4]
            end
        end
        (@nref 4 output i) = s
    end

In [27]:
output

1×4×4×3 Array{Real,4}:
[:, :, 1, 1] =
 -219  -606  90  -234

[:, :, 2, 1] =
 441  63  825  96

[:, :, 3, 1] =
 -288  -141  -234  -201

[:, :, 4, 1] =
 486  -249  -846  45

[:, :, 1, 2] =
 -801  -486  -1017  606

[:, :, 2, 2] =
 -507  -849  51  -114

[:, :, 3, 2] =
 -327  -258  -18  228

[:, :, 4, 2] =
 252  -669  42  180

[:, :, 1, 3] =
 -237  -132  1215  -174

[:, :, 2, 3] =
 -135  267  -477  -408

[:, :, 3, 3] =
 342  450  90  66

[:, :, 4, 3] =
 -336  765  114  -330